<a href="https://colab.research.google.com/github/KnzMcn/Machine-Learning/blob/main/Macan_ML_M4L1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTRUCTIONS:** Explore the process of building an artificial neural network for regression.

# Import the necessities

In [97]:
# import the libraries
import pandas as pd
import numpy as np

from keras.layers import Dense
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

from google.colab import files
import io

In [4]:
# import the dataset
uploaded = files.upload()
dataset = pd.read_csv(io.BytesIO(uploaded['Churn_Modelling.csv']))

Saving Churn_Modelling.csv to Churn_Modelling.csv


# Preprocessing

In [13]:
# view the whole dataset
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [82]:
# divide the features and labels
X = dataset.iloc[:, 3:-1].values
y = dataset['Exited'].values

In [83]:
# view the features
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [84]:
# view the labels
print(y)

[1 0 1 ... 1 1 0]


In [85]:
# encode categorical data
le = LabelEncoder()

#encode the gender variable
X[:, 2] = le.fit_transform(X[:, 2])

In [86]:
# one hot encode the geography column
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')

# do the one hot encoding
X = np.array(ct.fit_transform(X))

In [87]:
#check the changes
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [88]:
# split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [89]:
# scale the features
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Build the model

In [92]:
# instantiate the model
ann = Sequential()

# add the layers
ann.add(Dense(6, activation = 'relu'))
ann.add(Dense(6, activation = 'relu'))
ann.add(Dense(1, activation = 'sigmoid'))

#compile the layers
ann.compile(optimizer = 'adam', 
            loss = 'binary_crossentropy', 
            metrics = ['accuracy'])

In [ ]:
ann.summary()

In [94]:
#train the model
ann.fit(X_train, y_train, 
        batch_size = 32, 
        epochs = 20)

Epoch 1/20
250/250 [==============================] - 0s 2ms/step - loss: 0.4380 - accuracy: 0.8041
Epoch 2/20
250/250 [==============================] - 0s 2ms/step - loss: 0.4277 - accuracy: 0.8098
Epoch 3/20
250/250 [==============================] - 0s 2ms/step - loss: 0.4213 - accuracy: 0.8124
Epoch 4/20
250/250 [==============================] - 1s 2ms/step - loss: 0.4154 - accuracy: 0.8177
Epoch 5/20
250/250 [==============================] - 0s 2ms/step - loss: 0.4077 - accuracy: 0.8223
Epoch 6/20
250/250 [==============================] - 1s 2ms/step - loss: 0.3980 - accuracy: 0.8299
Epoch 7/20
250/250 [==============================] - 0s 2ms/step - loss: 0.3866 - accuracy: 0.8371
Epoch 8/20
250/250 [==============================] - 1s 2ms/step - loss: 0.3759 - accuracy: 0.8432
Epoch 9/20
250/250 [==============================] - 1s 2ms/step - loss: 0.3671 - accuracy: 0.8462
Epoch 10/20
250/250 [==============================] - 1s 2ms/step - loss: 0.3606 - accuracy: 0.8510

# Evaluation

In [95]:
#predict a with specified values
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 230ms/step
[[False]]


In [96]:
# predict using the test set
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [98]:
# evaluate using confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

# evaluate using accuracy score
accuracy_score(y_test, y_pred)

[[1534   61]
 [ 204  201]]


0.8675